# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

Pessoa(_Id_, Gender, Age)  
Medications_drug(_Code_, Name)  
Medications_use(Person_id, Drug_code, Days_use)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;CHE: Person_id (Pessoa)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;CHE: Drug_code (Medications_drug)  

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [2]:
%defaultDatasource jdbc:h2:mem:db

In [3]:
DROP TABLE IF EXISTS Pessoa;

CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../data/nhanes/demographic-person.csv');

In [4]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [5]:
DROP TABLE IF EXISTS Medications_use;
DROP TABLE IF EXISTS Medications_drug;

CREATE TABLE Medications_drug(
    Code VARCHAR(6),
    Name VARCHAR(100),
    PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('../data/nhanes/medications-drug.csv');

CREATE TABLE Medications_use(
    Person_id VARCHAR(5) NOT NULL,
    Drug_code VARCHAR(6) NOT NULL,
    Days_use INTEGER,
    FOREIGN KEY(Person_id)
        REFERENCES Pessoa(Id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY(Drug_code)
        REFERENCES Medications_drug(Code)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../data/nhanes/medications-use.csv');

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [6]:
SELECT * FROM Pessoa P
    WHERE P.Age > 60

b) Liste as idades das pessoas cadastradas (sem repetições)

In [7]:
SELECT DISTINCT P.Age FROM Pessoa P;

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [8]:
SELECT P.Id, P.Age, MU.Drug_code, MU.Days_use
    FROM Pessoa P, Medications_use MU
    WHERE P.Id = MU.Person_id;

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [9]:
SELECT P.Id, P.Age, MD.Name, MU.Days_use
    FROM Pessoa P, Medications_use MU, Medications_drug MD
    WHERE P.Id = MU.Person_id AND MD.Code = MU.Drug_code;

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [13]:
SELECT DISTINCT MD.Name
    FROM Medications_drug MD, Medications_use MU
    WHERE MD.Code = MU.Drug_code
        ORDER BY MD.Name;

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [14]:
SELECT DISTINCT MD.Name
    FROM Medications_drug MD, Medications_use MU
    WHERE MD.Code = MU.Drug_code AND MU.Days_use > 2000
        ORDER BY MD.Name;